In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install kaggle
import opendatasets as od
od.download('https://www.kaggle.com/datasets/joel771/melanomas7711')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
print(path)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Defaulting to user installation because normal site-packages is not writeable


100%|██████████| 228M/228M [03:35<00:00, 1.11MB/s] 


NameError: name 'path' is not defined

In [ ]:
import pandas as pd
import numpy as np 
import cv2
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, Conv2D,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.utils import class_weight
import os

from tensorflow.keras.applications import InceptionV3

In [ ]:
# meta/images paths
def map_imgs(meta_file_path, image_file_path):
    IMAGE_PATH = image_file_path
#     meta_file= '/kaggle/input/siim-isic-melanoma-classification/train.csv'
#     IMAGE_PATH = '/kaggle/input/siim-isic-melanoma-classification/jpeg/train'
    
    df = pd.read_csv(meta_file_path)

    # adding new column = filepath (complete image path)
    df['image_path'] = df['image_name'].map(lambda x:  os.path.join(IMAGE_PATH,x+'.jpg'))

    # mapping dictionary
    labelmap = {}
    for l in df.target.unique().tolist():
        if l == 1:
            labelmap[l] = 'melanoma'
        else:
            labelmap[l] = 'benign'

    # seperate list of image that are labelled = melanoma
    df_melanoma = df[df['target'] == 1]
    df_melanoma.reset_index(drop=True, inplace=True)

    # view 
    print(f"Found {df.groupby('target').count()['image_name'][1]} images that are labelled = melanoma and {df.groupby('target').count()['image_name'][0]} that are labelled = benign")
    return df

In [ ]:
df_train = map_imgs("/kaggle/input/siim-isic-melanoma-classification/train.csv","/kaggle/input/siim-isic-melanoma-classification/jpeg/train")
df_train= df_train[["target","image_path"]]
df_train["target"] = df_train["target"].astype(str)
df_train

In [ ]:
df_train = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')

In [ ]:
df_train.head()

In [ ]:
df_train['image_path'] = "/kaggle/input/siim-isic-melanoma-classification/jpeg/train/"+df_train['image_name']+".jpg"
df_train.head()

In [ ]:
df_input = df_train[['target','image_path']]
df_input["target"] = df_input["target"].astype(str)
df_input.head()

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

In [ ]:
train_generator = data_generator.flow_from_dataframe(df_input,
                                              target_size= (224, 224),
                                                     x_col='image_path',
                                                     y_col='target',
                                              batch_size = 128,
                                              subset = 'training',
                                              shuffle = True,
                                              class_mode ='binary')
val_generator = data_generator.flow_from_dataframe(df_input,
                                              target_size= (224, 224),
                                                   x_col='image_path',
                                                     y_col='target',
                                              batch_size = 128,
                                              subset = 'validation',
                                              shuffle = False,
                                              class_mode ='binary')

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes= np.unique(df_train["target"]),
                                                 y =df_train["target"])

In [ ]:
# convert numpy array to dictionary
class_weight = dict(enumerate(class_weights.flatten(), 0))
class_weight

In [ ]:
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))

In [ ]:
img_width = 224
img_height = 224
with tf.device(device_name):

    base_model = InceptionV3(
      weights='imagenet',
      include_top=False,
      input_shape=(img_width, img_height, 3))
    for layer in base_model.layers:
      layer.trainable = False

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))        # Create a TensorFlow model.
    
    opt = Adam(learning_rate=0.0001)

    model.compile(
    optimizer=opt,
    loss='binary_crossentropy',
    metrics=['accuracy'])

## GPU P100 Time

In [ ]:
%%time

from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(patience=10, verbose=1)

history = model.fit(
    train_generator,
    epochs=2,
    validation_data=val_generator, 
    callbacks=[early_stop],
    verbose=1)

## GPU TX2 Time

In [ ]:
%%time

from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(patience=10, verbose=1)

history = model.fit(
    train_generator,
    epochs=2,
    validation_data=val_generator, 
    callbacks=[early_stop],
    verbose=1)
     